## Obtain ogap and speedup for the trained models

In [1]:
import numpy as np
import itertools
from multiprocessing import Pool

from bb import solve_bb, DefaultBranchingPolicy
from bb import BBenv as Environment
import os
import pickle
import time
from pathlib import Path
from single_group_as_bm.observation import Observation

def solve_bb_pool(arguments):
    instance, max_ant = arguments
    return solve_bb(instance, max_ant=max_ant)


def solve_ml_pool(arguments):
    instance, w_optimal, optimal_objective, file_count, policy_filepath, max_ant, mask_heuristics, use_heuristics = arguments

    env = Environment(observation_function=Observation, epsilon=0.002)
    env.set_node_select_policy(node_select_policy_path=policy_filepath, policy_type='gnn')
    
    if use_heuristics:
        env.set_heuristic_solutions(mask_heuristics)

    env.reset(instance, max_ant=max_ant,  oracle_opt=w_optimal)

    branching_policy = DefaultBranchingPolicy()
    t1 = time.time()

    timestep = 0
    done = False
    time_taken = 0
    sum_label = 0
    node_counter = 0

    while timestep < 1000 and len(env.nodes)>0 and not done: 
        print('timestep {}'.format(timestep))
    
        env.fathom_nodes()
        if len(env.nodes) == 0:
            break
        node_id, node_feats, label = env.select_node()
#         print('observation instance', isinstance(node_feats, Observation))

        if len(env.nodes) == 0:
            break
        time_taken += time.time()-t1
        sum_label += label  
        node_counter += 1
        t1 = time.time()

        # print("Selection Time {}".format(time.time()-t1))
        # t1 = time.time()

        prune_node = env.prune(node_feats)
        # prune_node = False
        
        # print("Prune Decision Time {}".format(time.time()-t1))
        # t1 = time.time()

        if prune_node:
            env.delete_node(node_id)
            continue
        else:
            branching_var = branching_policy.select_variable(node_feats, env.action_set_indices)
            # try:
            #     done = env.push_children(branching_var, node_id)
            # except:
            #     print("exception occured")
            #     break
            done = env.push_children(branching_var, node_id)
        
        # print("Push children Time {}".format(time.time()-t1))
        
        timestep = timestep+1

    # ml = np.linalg.norm(env.W_incumbent, 'fro')**2
    ml = env.global_U
    ogap = ((ml - optimal_objective)/optimal_objective)*100
    time_taken += time.time() - t1
    print('instance result', timestep, ogap, time_taken, sum_label, optimal_objective, ml)
    return timestep, ogap, time_taken, sum_label/node_counter


In [19]:
import numpy as np
import itertools
from multiprocessing import Pool

import os
import pickle
import time
from pathlib import Path

N = 6
M = 12
max_ant = 2
np.random.seed(seed = 100)
num_egs = 30

ftpl2 = []
nonftpl = []

folder_path = '/scratch/sagar/Projects/combopt/branch-and-bound-ml/single_group_as_bm/validation_set/N={},M={},L={}'.format(N,M,max_ant)

if not os.path.isdir(folder_path):
    Path(folder_path).mkdir(exist_ok=True)

result_filepath = os.path.join(folder_path, 'egs_set_valid_30.pkl') 

MODEL_FOLDERPATH = '/scratch/sagar/Projects/combopt/branch-and-bound-ml/single_group_as_bm/trained_models/N={},M={},L={}'.format(N,M,max_ant)

num_nodes_eval = 0
time_total = 0
obj_bb = 0
obj_omar = 0
obj_opt = 0
t1 = time.time()

instances = np.random.randn(num_egs, 2, N, M)

######### ORACLE method #####################

# t1 = time.time()
# arguments_oracle = list(zip(list(instances), [max_ant]*num_egs))
# # with Pool(10) as p:
# #     out_oracle = p.map(solve_bb_pool, arguments_oracle)
# #     print('pool ended')
# # oracle_time = time.time() - t1
# out_oracle =[]
# for i in range(num_egs):
#     print('eg {}'.format(i))
#     output = solve_bb_pool(arguments_oracle[i])
#     out_oracle.append(output)


# optimal_solution_z_list = [out_oracle[i][0][0] for i in range(len(out_oracle))]
# optimal_solution_w_list = [out_oracle[i][0][1] for i in range(len(out_oracle))]

# optimal_objective_list = [out_oracle[i][1] for i in range(len(out_oracle))]
# oracle_timesteps = np.mean([out_oracle[i][2] for i in range(len(out_oracle))])
# oracle_time = np.mean([out_oracle[i][3] for i in range(len(out_oracle))])


# data = (instances, optimal_solution_z_list,  optimal_solution_w_list, optimal_objective_list, oracle_timesteps, oracle_time)
# with open(result_filepath, 'wb') as handle:
#     pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)    

########### LOAD data ################################################

with open(result_filepath, 'rb') as handle:
    data = pickle.load(handle)
    instances, optimal_solution_z_list, optimal_solution_w_list, optimal_objective_list, oracle_timesteps, oracle_time = data

########## OMAR'S Method #######################################
mask_omar = [0]*N
# omar_time = 0
# for i in range(num_egs):
#     H = instances[i,0,:,:] + 1j*instances[i,1,:,:]
#     obj_bb += optimal_objective_list[i]
#     t1 = time.time()
#     obj, mask = as_omar(H, max_ant=max_ant)
#     omar_time += time.time() - t1
#     obj_omar += obj
#     mask_omar.append(mask)
# omar_time = omar_time/num_egs
# print(omar_time)

########### GNN MODEL ###################################

model_ids = [2,5]

### FTPL ###
for i in model_ids:
    MODEL_FILEPATH = os.path.join(MODEL_FOLDERPATH, 'gnn_ftpl3_iter_{}'.format(i))
    t1 = time.time()
    arguments_ml = list(zip(list(instances), 
                            list(zip(optimal_solution_z_list, optimal_solution_w_list)), 
                            optimal_objective_list, 
                            range(num_egs), 
                            [MODEL_FILEPATH]*num_egs, 
                            [max_ant]*num_egs, 
                            [mask_omar]*num_egs, 
                            [False]*num_egs))
    
    out_ml =[]
    for j in range(num_egs):
        print('eg {}'.format(i))
        try:
            output = solve_ml_pool(arguments_ml[j])
            out_ml.append(output)
        except:
            pass
#     with Pool(10) as p:
#         print('started pool')
#         out_ml = p.map(solve_ml_pool, arguments_ml)
    
    avg_ml_ogap = np.mean(np.array([out_ml[i][1] for i in range(len(out_ml))]))
    avg_ml_steps = np.mean(np.array([out_ml[i][0] for i in range(len(out_ml))]))
    ml_time = np.mean(np.array([out_ml[i][2] for i in range(len(out_ml))]))
    ftpl2.append((avg_ml_ogap, avg_ml_steps, ml_time))


eg 2
policy path /scratch/sagar/Projects/combopt/branch-and-bound-ml/single_group_as_bm/trained_models/N=6,M=12,L=2/gnn_ftpl3_iter_2
timestep 0
timestep 1
timestep 2
timestep 3
timestep 4
timestep 5
relaxed: not optimal, may be infeasible
antennas: not optimal, may be infeasible
timestep 6
timestep 6
timestep 7
timestep 7
timestep 8
timestep 9
timestep 10
antennas: not optimal, may be infeasible
timestep 11
relaxed: not optimal, may be infeasible
antennas: not optimal, may be infeasible
timestep 12
antennas: not optimal, may be infeasible
timestep 13
relaxed: not optimal, may be infeasible
timestep 14
timestep 14
timestep 14
relaxed: not optimal, may be infeasible
timestep 15
timestep 15
timestep 16
relaxed: not optimal, may be infeasible
antennas: not optimal, may be infeasible
timestep 17
relaxed: not optimal, may be infeasible
timestep 18
antennas: not optimal, may be infeasible
timestep 19
timestep 19
timestep 19
relaxed: not optimal, may be infeasible
timestep 20
timestep 21
times

timestep 92
antennas: not optimal, may be infeasible
timestep 93
antennas: not optimal, may be infeasible
timestep 94
antennas: not optimal, may be infeasible
timestep 95
antennas: not optimal, may be infeasible
timestep 96
antennas: not optimal, may be infeasible
timestep 97
antennas: not optimal, may be infeasible
timestep 98
antennas: not optimal, may be infeasible
timestep 99
antennas: not optimal, may be infeasible
timestep 100
antennas: not optimal, may be infeasible
timestep 101
antennas: not optimal, may be infeasible
timestep 102
antennas: not optimal, may be infeasible
timestep 103
antennas: not optimal, may be infeasible
timestep 104
antennas: not optimal, may be infeasible
timestep 105
antennas: not optimal, may be infeasible
timestep 106
antennas: not optimal, may be infeasible
timestep 107
antennas: not optimal, may be infeasible
timestep 108
antennas: not optimal, may be infeasible
timestep 109
antennas: not optimal, may be infeasible
timestep 110
antennas: not optimal, 

timestep 31
timestep 32
timestep 33
timestep 33
timestep 33
relaxed: not optimal, may be infeasible
antennas: not optimal, may be infeasible
timestep 34
timestep 34
relaxed: not optimal, may be infeasible
timestep 35
timestep 35
timestep 35
relaxed: not optimal, may be infeasible
relaxed: not optimal, may be infeasible
timestep 36
relaxed: not optimal, may be infeasible
antennas: not optimal, may be infeasible
antennas: not optimal, may be infeasible
timestep 37
timestep 37
timestep 37
timestep 37
timestep 37
timestep 37
timestep 37
timestep 38
timestep 38
timestep 38
timestep 38
timestep 38
timestep 38
timestep 38
antennas: not optimal, may be infeasible
timestep 39
relaxed: not optimal, may be infeasible
antennas: not optimal, may be infeasible
antennas: not optimal, may be infeasible
timestep 40
timestep 40
timestep 40
timestep 40
timestep 40
timestep 40
timestep 40
timestep 40
timestep 40
timestep 40
timestep 40
timestep 41
timestep 41
timestep 41
timestep 41
timestep 41
timestep 4

timestep 1
timestep 2
timestep 3
timestep 4
timestep 5
timestep 5
timestep 6
timestep 7
timestep 7
timestep 7
antennas: not optimal, may be infeasible
timestep 8
relaxed: not optimal, may be infeasible
timestep 9
antennas: not optimal, may be infeasible
timestep 10
relaxed: not optimal, may be infeasible
timestep 11
timestep 12
timestep 12
timestep 12
antennas: not optimal, may be infeasible
timestep 13
timestep 13
timestep 13
timestep 14
timestep 14
relaxed: not optimal, may be infeasible
timestep 15
timestep 15
timestep 15
timestep 15
antennas: not optimal, may be infeasible
timestep 16
relaxed: not optimal, may be infeasible
timestep 17
timestep 17
timestep 17
timestep 17
timestep 17
timestep 17
timestep 17
timestep 18
timestep 18
timestep 18
timestep 18
timestep 18
timestep 18
timestep 18
relaxed: not optimal, may be infeasible
relaxed: not optimal, may be infeasible
timestep 19
timestep 19
timestep 19
timestep 19
timestep 19
relaxed: not optimal, may be infeasible
relaxed: not opt

timestep 1
timestep 2
timestep 3
timestep 4
timestep 5
timestep 6
timestep 7
timestep 8
timestep 9
antennas: not optimal, may be infeasible
antennas: not optimal, may be infeasible
timestep 10
relaxed: not optimal, may be infeasible
timestep 11
timestep 11
timestep 11
timestep 11
timestep 11
timestep 12
timestep 12
relaxed: not optimal, may be infeasible
antennas: not optimal, may be infeasible
timestep 13
timestep 13
timestep 14
timestep 14
timestep 15
antennas: not optimal, may be infeasible
antennas: not optimal, may be infeasible
timestep 16
timestep 17
timestep 17
timestep 17
timestep 17
timestep 17
timestep 17
timestep 17
timestep 17
relaxed: not optimal, may be infeasible
relaxed: not optimal, may be infeasible
timestep 18
relaxed: not optimal, may be infeasible
timestep 19
timestep 19
relaxed: not optimal, may be infeasible
timestep 20
timestep 20
timestep 20
timestep 20
timestep 20
timestep 20
relaxed: not optimal, may be infeasible
relaxed: not optimal, may be infeasible
time

...

timestep 115
antennas: not optimal, may be infeasible
timestep 116
antennas: not optimal, may be infeasible
timestep 117
antennas: not optimal, may be infeasible
timestep 118
antennas: not optimal, may be infeasible
timestep 119
antennas: not optimal, may be infeasible
timestep 120
antennas: not optimal, may be infeasible
timestep 121
antennas: not optimal, may be infeasible
timestep 122
antennas: not optimal, may be infeasible
timestep 123
antennas: not optimal, may be infeasible
timestep 124
antennas: not optimal, may be infeasible
timestep 125
antennas: not optimal, may be infeasible
timestep 126
antennas: not optimal, may be infeasible
timestep 127
antennas: not optimal, may be infeasible
timestep 128
antennas: not optimal, may be infeasible
timestep 129
antennas: not optimal, may be infeasible
timestep 130
antennas: not optimal, may be infeasible
timestep 131
antennas: not optimal, may be infeasible
timestep 132
antennas: not optimal, may be infeasible
timestep 133
antennas: not o

asserting False ((array([1., 1., 1., 1., 1., 1.]), array([0., 1., 0., 0., 1., 0.])), (array([0.        , 1.30081571, 3.86563159, 0.        , 3.06182565,
       3.58337912, 3.58337912, 3.14159265, 0.        , 1.15355355,
       0.        , 0.        ]), array([6.28318531, 1.30695163, 3.87176751, 6.28318531, 3.06796158,
       3.58951504, 3.59565097, 3.14772858, 6.28318531, 1.15968948,
       6.28318531, 6.28318531]))) [1. 1. 1. 1. 1. 1.] [0. 1. 0. 0. 1. 0.] [0.         1.30081571 3.86563159 0.         3.06182565 3.58337912
 3.58337912 3.14159265 0.         1.15355355 0.         0.        ] [6.28318531 1.30695163 3.87176751 6.28318531 3.06796158 3.58951504
 3.60792281 3.14772858 6.28318531 1.15968948 6.28318531 6.28318531] [[[ 3.16169343e-01 -1.78885720e-01 -1.07678501e+00 -1.13851972e+00
   -1.30576759e-01 -3.44793847e-01 -6.60735958e-01 -4.94910091e-02
   -3.71024354e-01 -1.39017608e-01  2.99215697e-01 -3.83108742e-01]
  [ 1.07022049e+00 -2.57350094e-01 -2.28778818e-01 -1.00117964e-01


antennas: not optimal, may be infeasible
timestep 126
antennas: not optimal, may be infeasible
timestep 127
antennas: not optimal, may be infeasible
timestep 128
antennas: not optimal, may be infeasible
timestep 129
antennas: not optimal, may be infeasible
timestep 130
antennas: not optimal, may be infeasible
timestep 131
antennas: not optimal, may be infeasible
timestep 132
antennas: not optimal, may be infeasible
timestep 133
antennas: not optimal, may be infeasible
timestep 134
antennas: not optimal, may be infeasible
timestep 135
antennas: not optimal, may be infeasible
timestep 136
antennas: not optimal, may be infeasible
timestep 137
antennas: not optimal, may be infeasible
timestep 138
antennas: not optimal, may be infeasible
timestep 139
antennas: not optimal, may be infeasible
timestep 140
antennas: not optimal, may be infeasible
timestep 141
antennas: not optimal, may be infeasible
timestep 142
antennas: not optimal, may be infeasible
timestep 143
antennas: not optimal, may b

timestep 10
timestep 10
timestep 10
timestep 10
relaxed: not optimal, may be infeasible
relaxed: not optimal, may be infeasible
timestep 11
timestep 11
timestep 11
relaxed: not optimal, may be infeasible
timestep 12
timestep 12
timestep 12
timestep 12
timestep 12
timestep 12
timestep 12
timestep 12
timestep 12
timestep 12
timestep 12
timestep 12
timestep 12
timestep 12
timestep 12
timestep 12
timestep 12
timestep 12
timestep 12
timestep 12
instance result 12 70.28216149993462 2.5175576210021973 3 0.9790163926029347 1.6670902747629632
eg 5
policy path /scratch/sagar/Projects/combopt/branch-and-bound-ml/single_group_as_bm/trained_models/N=6,M=12,L=2/gnn_ftpl3_iter_5
timestep 0
timestep 1
timestep 2
timestep 3
timestep 4
timestep 4
relaxed: not optimal, may be infeasible
timestep 5
timestep 5
timestep 5
timestep 5
timestep 6
timestep 6
timestep 6
timestep 6
timestep 6
timestep 6
timestep 6
timestep 6
timestep 6
relaxed: not optimal, may be infeasible
relaxed: not optimal, may be infeasibl

timestep 1
timestep 2
timestep 3
timestep 4
timestep 5
timestep 5
timestep 5
relaxed: not optimal, may be infeasible
timestep 6
timestep 6
timestep 6
timestep 6
antennas: not optimal, may be infeasible
timestep 7
timestep 7
antennas: not optimal, may be infeasible
antennas: not optimal, may be infeasible
timestep 8
timestep 9
timestep 9
timestep 9
timestep 9
timestep 9
timestep 9
timestep 9
timestep 9
timestep 9
timestep 9
timestep 9
instance result 9 155.38569211325245 1.0575048923492432 3 0.6044305683557735 1.5436291903394574
eg 5
policy path /scratch/sagar/Projects/combopt/branch-and-bound-ml/single_group_as_bm/trained_models/N=6,M=12,L=2/gnn_ftpl3_iter_5
timestep 0
timestep 1
timestep 2
timestep 3
timestep 3
timestep 4
timestep 4
timestep 4
relaxed: not optimal, may be infeasible
timestep 5
timestep 5
timestep 5
timestep 6
timestep 6
timestep 6
timestep 6
timestep 6
timestep 6
timestep 6
timestep 6
timestep 6
timestep 6
timestep 6
instance result 6 43.90482950928315 1.5968394279479

In [20]:
ftpl2

[(43.883036704148616, 41.10344827586207, 7.747441020505182),
 (70.46124661383082, 7.7, 1.7411332607269288)]

In [ ]:
orac

In [3]:
ftpl[]

[(6.992548570957692, 38.5, 30.74087756474813),
 (5.840530491918151, 44.9, 33.824991655349734),
 (7.692282614626715, 23.0, 19.994152220090232),
 (8.23608836499299, 24.766666666666666, 21.612661441167194),
 (9.861599450852633, 23.3, 19.912328481674194),
 (10.525642705196267, 23.5, 20.082302220662434),
 (18.165140841459962, 12.366666666666667, 13.318800338109334),
 (12.564284019296675, 22.766666666666666, 19.441139356295267),
 (3.5836367171592363, 39.7, 33.21012505690257),
 (19.26675039954734, 17.3, 16.141164032618203),
 (15.734027405856155, 18.1, 17.222631549835206),
 (21.979397986533286, 9.933333333333334, 10.398924922943115),
 (1.2945145302247003, 49.4, 37.595393999417624),
 (21.986216525788286, 15.433333333333334, 14.52993000348409),
 (19.308200564496197, 17.2, 16.567367951075237),
 (13.193023028378166, 20.166666666666668, 19.062581268946328)]

In [24]:
len(list(zip(optimal_solution_z_list, optimal_solution_w_list)))

30

In [9]:
ftpl2

[(18.165140841459962, 12.366666666666667, 0.6667248884836833),
 (12.564284019296675, 22.766666666666666, 0.7918498357137044),
 (21.979397986533286, 9.933333333333334, 0.5657983223597208),
 (1.2945145302247003, 49.4, 1.3216330448786417)]

In [15]:
ftpl2

[]

In [21]:
time_speedup = []

for i in range(len(ftpl2)):
    time_speedup.append(oracle_time/ftpl2[i][2])

In [22]:
time_speedup

[3.8486505856334716, 17.1251644508125]

In [5]:
oracle_timesteps

40.0

In [27]:
len(arguments_ml)

30

In [15]:
avg_ml_ogap

nan

In [10]:
oracle_time

1.3494306802749634

In [11]:
list(zip(optimal_solution_z_list, optimal_solution_w_list))

[(array([1., 0., 1., 1.]),
  array([-0.3600913 -0.10700754j,  0.        +0.j        ,
          0.38377091-0.21161514j,  0.07574091+0.07189458j])),
 (array([1., 0., 1., 1.]),
  array([0.17070511+0.34609164j, 0.        +0.j        ,
         0.09020742-0.19837645j, 0.20666546-0.21333487j])),
 (array([1., 1., 0., 1.]),
  array([-1.25401403e-01-0.27718803j,  2.04772816e-02-0.29069566j,
          0.00000000e+00+0.j        , -2.39824757e-05-0.39711481j])),
 (array([0., 1., 1., 1.]),
  array([ 0.        +0.j        , -0.14411751+0.04834938j,
          0.16589445-0.38702259j,  0.1601583 -0.13994473j])),
 (array([1., 1., 0., 1.]),
  array([-0.21939257-0.09798992j, -0.40447263+0.07248383j,
          0.        +0.j        , -0.06682381-0.04083789j])),
 (array([1., 1., 0., 1.]),
  array([-0.38099521+0.19787626j, -0.13149595+0.13530164j,
          0.        +0.j        ,  0.01312272-0.03724939j])),
 (array([1., 0., 1., 1.]),
  array([0.23206503+0.05249169j, 0.        +0.j        ,
         0.40809

In [8]:
len(out_oracle[0])

4

In [16]:

optimal_solution_z_list = [out_oracle[i][0][0] for i in range(len(out_oracle))]
optimal_solution_w_list = [out_oracle[i][0][1] for i in range(len(out_oracle))]

optimal_objective_list = [out_oracle[i][1] for i in range(len(out_oracle))]
oracle_timesteps = np.mean([out_oracle[i][2] for i in range(len(out_oracle))])
oracle_time = np.mean([out_oracle[i][3] for i in range(len(out_oracle))])

data = (instances, optimal_solution_z_list,  optimal_solution_w_list, optimal_objective_list, oracle_timesteps, oracle_time)
with open(result_filepath, 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)    

In [4]:
from pathlib import Path

list(Path('data/dagger/N=4,M=3,L=3/oracle_data').glob('*.pkl'))


[PosixPath('data/dagger/N=4,M=3,L=3/oracle_data/a.pkl')]